In [89]:
###Dependecies
import sqlite3
import pandas as pd
from io import BytesIO
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import numpy as np
from config import skey
import json
import pprint as pp
from collections import OrderedDict 


In [18]:
pd.options.display.float_format = '{:.2f}'.format


In [19]:
user_data = pd.read_csv('data/100k_ids.csv')
user_data.head()

,steamid,timecreated,gameserverip,cityid,loccityid,loccountrycode,locstatecode
0,nan,NaN,NaN,nan,nan,NaN,NaN
1,76561200000000000.00,9/10/2003 20:58,NaN,nan,43754.00,SE,26
2,nan,NaN,NaN,nan,nan,NaN,NaN
3,76561200000000000.00,9/10/2003 21:53,NaN,nan,nan,US,NaN
4,nan,NaN,NaN,nan,nan,NaN,NaN


In [14]:
user_ids = user_data.dropna(how='all')
user_ids.reset_index(drop=True, inplace=True)
user_ids = user_ids.drop(columns=['timecreated', 'gameserverip', 'cityid', 'loccityid'])
user_ids = user_ids.rename(columns={'steamid': 'Steam ID',
                                    'loccountrycode': 'Country',
                                    'locstatecode': 'State'})
user_ids.head()

,Steam ID,Country,State
0,7.656120e+16,SE,26
1,7.656120e+16,US,NaN
2,7.656120e+16,CA,BC
3,7.656120e+16,AU,VIC
4,7.656120e+16,US,WA


In [29]:
steam_ids = user_ids['Steam ID'].tolist()
print(len(steam_ids))

99999


In [17]:
#user_ids['Steam ID'] = user_ids['Steam ID'].apply(lambda x: '{:.2f}'.format(x))
user_ids.head()

,Steam ID,Country,State
0,76561200000000000.00,SE,26
1,76561200000000000.00,US,NaN
2,76561200000000000.00,CA,BC
3,76561200000000000.00,AU,VIC
4,76561200000000000.00,US,WA


In [140]:
temp_steam_id = 76561198048037824
player_summary_url = f"http://api.steampowered.com/ISteamUser/GetPlayerSummaries/v0002/?key={skey}&steamids={temp_steam_id}"
response = requests.get(player_summary_url).json()
#print(steam_ids[0])
#print(response)

In [141]:
print(json.dumps(response, indent=4, sort_keys=True))

{
    "response": {
        "players": [
            {
                "avatar": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/76/76bc1af0c42b7f9cf13de0af3d5d6aefe72bdbb3.jpg",
                "avatarfull": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/76/76bc1af0c42b7f9cf13de0af3d5d6aefe72bdbb3_full.jpg",
                "avatarhash": "76bc1af0c42b7f9cf13de0af3d5d6aefe72bdbb3",
                "avatarmedium": "https://steamcdn-a.akamaihd.net/steamcommunity/public/images/avatars/76/76bc1af0c42b7f9cf13de0af3d5d6aefe72bdbb3_medium.jpg",
                "communityvisibilitystate": 3,
                "lastlogoff": 1615549796,
                "loccountrycode": "US",
                "locstatecode": "IL",
                "personaname": "GAGAMBA STYLE",
                "personastate": 0,
                "personastateflags": 0,
                "primaryclanid": "103582791429521408",
                "profilestate": 1,
                "profileurl": "

In [142]:
#print(response['response']['players'][0].get("personaname"))
temp_user_dict = {"user_steam_id" : temp_steam_id}

In [143]:

temp_user_dict.update({"original_account_name" : response['response']['players'][0].get("realname")})
temp_user_dict.update({"current_account_name" : response['response']['players'][0].get("personaname")})
temp_user_dict.update({"profile_url" : response['response']['players'][0].get("profileurl")})


In [144]:
pp.pprint(temp_user_dict)

{'current_account_name': 'GAGAMBA STYLE',
 'original_account_name': 'Uber_Pro',
 'profile_url': 'https://steamcommunity.com/id/uber_pro/',
 'user_steam_id': 76561198048037824}


In [66]:
owned_games_url = f"http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={skey}&steamid=76561198048037824&format=json"
game_list = requests.get(owned_games_url).json()
print(json.dumps(game_list, indent=2))

{
  "response": {
    "game_count": 14,
    "games": [
      {
        "appid": 400,
        "playtime_forever": 0,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 10180,
        "playtime_forever": 330,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 10190,
        "playtime_forever": 12154,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 42680,
        "playtime_forever": 416,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 42690,
        "playtime_forever": 3224,
        "playtime_windows_forever": 0,
        "playtime_mac_forever": 0,
        "playtime_linux_forever": 0
      },
      {
        "appid": 2

In [68]:
print(game_list["response"]["games"][0])

{'appid': 400, 'playtime_forever': 0, 'playtime_windows_forever': 0, 'playtime_mac_forever': 0, 'playtime_linux_forever': 0}


In [135]:
user_dictionary = OrderedDict({"steamid": 69,
                               "current_player_name": "all I do is code now",
                               "total_games_played" : {},
                               "user_game_data" : {}
                              })

user_game_stats = {}
play_count = 0

In [151]:

for i in range(game_list["response"]["game_count"]):
    
    #initialize temporary variables
    appid = 0
    hours_played = 0
    
    if game_list["response"]["games"][i].get("playtime_forever") != 0:
        appid = game_list["response"]["games"][i].get("appid")
        hours_played = game_list["response"]["games"][i].get("playtime_forever")
        #function to get appid actual name
        play_count += 1
        
#         dict_name = f"game{play_count}"
         
        user_game_stats.update({ f"game{play_count}" : {"appid" : appid,
                                                        "hours played" : "{:.2f}".format(hours_played/60)
                                                       }
                               })
    #print(user_game_stats)
        
user_dictionary.update({"total_games_played" : play_count})
user_dictionary.update({"user_game_data" : user_game_stats})



In [137]:
#pp.pprint(user_game_stats)
pp.pprint(user_dictionary)

OrderedDict([('steamid', 69),
             ('current_player_name', 'all I do is code now'),
             ('total_games_played', 8),
             ('user_game_data',
              {'game1': {'appid': 10180, 'hours played': '5.50'},
               'game2': {'appid': 10190, 'hours played': '202.57'},
               'game3': {'appid': 42680, 'hours played': '6.93'},
               'game4': {'appid': 42690, 'hours played': '53.73'},
               'game5': {'appid': 205790, 'hours played': '0.52'},
               'game6': {'appid': 221380, 'hours played': '4.47'},
               'game7': {'appid': 550, 'hours played': '0.37'},
               'game8': {'appid': 544730, 'hours played': '0.03'}})])


In [5]:
game_news_url = f"http://api.steampowered.com/ISteamNews/GetNewsForApp/v0002/?appid=4000&count=3&maxlength=300&format=json"
game_news_test = requests.get(game_news_url).json()
base_appid_url = "https://steamdb.info/app/"
#print(json.dumps(game_news_test, indent=2))

In [7]:
# Retrieve page with the requests module
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/89.0.4389.23/chromedriver_win32.zip


[WDM] - Driver has been saved in cache [C:\Users\pwitw\.wdm\drivers\chromedriver\win32\89.0.4389.23]


In [33]:
browser.visit(base_appid_url + "10190")
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [34]:
game_title = soup.find('h1').get_text()
print(game_title)

Call of Duty: Modern Warfare 2 - Multiplayer


In [35]:
#test friendslist api call
target_userid = "76561198048037824"
friend_list_url = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={skey}&steamid={target_userid}&relationship=friend"

friends_list_test = requests.get(friend_list_url).json()
print(json.dumps(friends_list_test, indent=2))

{
  "friendslist": {
    "friends": [
      {
        "steamid": "76561197960385707",
        "relationship": "friend",
        "friend_since": 1391513349
      },
      {
        "steamid": "76561197961453204",
        "relationship": "friend",
        "friend_since": 1401181835
      },
      {
        "steamid": "76561197962793311",
        "relationship": "friend",
        "friend_since": 1358128159
      },
      {
        "steamid": "76561197963007752",
        "relationship": "friend",
        "friend_since": 1379723775
      },
      {
        "steamid": "76561197963921615",
        "relationship": "friend",
        "friend_since": 1342867333
      },
      {
        "steamid": "76561197966111577",
        "relationship": "friend",
        "friend_since": 1427995002
      },
      {
        "steamid": "76561197967404721",
        "relationship": "friend",
        "friend_since": 1395817709
      },
      {
        "steamid": "76561197968181086",
        "relationship": "friend"

In [53]:
#print(len(friends_list_test["friendslist"]["friends"]))
current_friendslist_length = len(friends_list_test["friendslist"]["friends"])
print(friends_list_test["friendslist"]["friends"][0].get("steamid"))

76561197960385707


In [54]:
steamid_collection = []
for i in range(current_friendslist_length):
    steamid_collection.append(friends_list_test["friendslist"]["friends"][i].get("steamid"))

In [60]:
len(steamid_collection) == len(set(steamid_collection))

True

In [65]:
target_userid = steamid_collection[5]
friend_list_url = f"http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={skey}&steamid={target_userid}&relationship=friend"

friends_list_test = requests.get(friend_list_url).json()
print(json.dumps(friends_list_test, indent=2))

{}


In [64]:
print(steamid_collection[5])


76561197966111577


In [146]:
testing = str(steamid_collection[5])
print(testing)

76561197966111577
